Notebook com ETL responsável por facrre

In [0]:
s3_path = "s3://case-tecnico-ifood/raw/{}/"
data_types = ["yellow", "green", "fhv", "fhvhv"]
catalog = "ifood_case"
schema = "bronze"

In [0]:
for data_type in data_types:
    date_folders = dbutils.fs.ls(s3_path.format(data_type))
    filtered_folders = [folder.path for folder in date_folders if "2023-01" not in folder.path]

    df = spark.read.parquet(*filtered_folders)
    if data_type == "fhvhv":
        df = df.repartition(8)
    else:
        df = df.coalesce(1)
    df.write.format("delta").mode("overwrite").saveAsTable(f"{catalog}.{schema}.{data_type}_after_202301")